# Räumliche Verknüpfung der Niederschlagsstationen mit den jeweiligen Abflussstationen

### Transformieren der Koordinaten
In diesem Block werden die Koordinaten der Niederschlagsstationen vom WGS84-Koordinatensystem (EPSG:4326) in das ETRS89-System (EPSG:25832) transformiert. Ziel ist es, für Abfluss- und Niederschlagsstationen ein einheitliches Koordinatensystem zu verwenden, um diese später räumlich miteinander verknüpfen zu können. Da die Koordinaten innerhalb einer Niederschlagsstation konstant sind, werden lediglich die ersten Zeilen jeder Datei eingelesen. Das daraus resultierende DataFrame enthält jeweils eine Zeile pro Niederschlagsstation mit den Spalten "Stations ID", "Stationsname", "Ostwert (ETRS89)", "Nordwert (ETRS89)" und wird als "Koordinaten der Niederschlagsstationen.csv" gespeichert.

In [13]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pathlib import Path
import glob
from pyproj import Transformer

waters = pd.read_csv("Endgültige_Auswahl_der_Gewässerstationen.csv", sep=",", encoding="utf-8")

precipitation_files = glob.glob("Datensätze/DWD/Filtered-Enhanced-Ba-Wü-Messnetze und Parameternetze-stündlich/*/*_standardised.csv")

# Transform WGS84 System from the DWD precipitation Data sets to ETRS89 System because its easier to calculate with distances afterwards.
transformer = Transformer.from_crs("EPSG:4326", "EPSG:25832", always_xy=True)

# transform just the standardised files. If a Data set is empty after filtering there was no respective standardised file.
def transform_wgs84_to_etrs89():
    rows = []
    for p in precipitation_files:
        df = pd.read_csv(p, encoding="utf-8", nrows=3)
        station_id = df["Stations ID"].iloc[0]
        station_name = df["Stationsname"].iloc[0]
        lat = df["Geographische Breite"].iloc[0]
        lon = df["Geographische Länge"].iloc[0]
        x, y = transformer.transform(lon, lat)

        rows.append({
            "Stations ID": station_id, "Stationsname": station_name, "Ostwert (ETRS89)": x, "Nordwert (ETRS89)": y
        })
    out = pd.DataFrame(rows).drop_duplicates(subset=["Stations ID"])

    out = out[["Stations ID", "Stationsname", "Ostwert (ETRS89)", "Nordwert (ETRS89)"]]
    out.to_csv("Koordinaten der Niederschlagsstationen.csv", index=False, encoding="utf-8")

    print("Transformed the coordinates and stored in table in 'Koordinaten der Niderschlagsstationen.csv'")

transform_wgs84_to_etrs89()



Transformed the coordinates and stored in table in 'Koordinaten der Niderschlagsstationen.csv'


### Verknüpfung der Stationen
In diesem Abschnitt werden die beiden Dateien "Endgültige_Auswahl_der_Gewässerstationen.csv" und "Koordinaten der Niederschlagsstationen.csv" eingelesen. Für jede Abflussstation wird anschließend überprüft, welche Niederschlagsstationen sich innerhalb eines Radius von 20 000 Metern befinden. Der Abstand wird dabei als euklidische Distanz im ETRS89-Koordinatensystem berechnet. 
Alle Niederschlagsstationen, deren entfernung unterhalb dieser 20 000 Meter liegt, werden über ihre Stations ID einer Liste zugeordnet. Das daraus resultierende DataFrame enthält die Spalten "Gewässer", "Gewässerstation", "Matched precipitation stations" (Die jeweilige Liste mit Stations IDs) und "Sum" (Anzahl der zugeordneten Stations IDs). Das DataFrame wird abschließend als "Matched coordinates.csv" gespeichert.

In [20]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pathlib import Path
import glob
from pyproj import Transformer
import math

def matching_the_stations():
    waters_coordinates = pd.read_csv("Endgültige_Auswahl_der_Gewässerstationen.csv", sep=",", encoding="utf-8")
    precipitation_coordinates = pd.read_csv("Koordinaten der Niederschlagsstationen.csv", sep=",", encoding="utf-8")
    
    RADIUS_IN_METER = 20_000
    
    matched = []
    for i in range(len(waters_coordinates)):
        water_row = waters_coordinates.iloc[i]
        water_x = float(water_row["Ostwert (ETRS89)"])
        water_y = float(water_row["Nordwert (ETRS89)"])

        precipitation_stations = []
        for j in range(len(precipitation_coordinates)):
            precipitation_row = precipitation_coordinates.iloc[j]
            precipitation_x = float(precipitation_row["Ostwert (ETRS89)"])
            precipitation_y = float(precipitation_row["Nordwert (ETRS89)"])

            distance = math.hypot(precipitation_x - water_x, precipitation_y - water_y)

            if distance <= RADIUS_IN_METER:
                precipitation_stations.append(str(precipitation_row["Stations ID"]))

        
            
        matched.append({
            "Gewässer": water_row["Gewässer"], "Gewässerstation": water_row["Stationsname"], 
            "Matched precipitation stations": precipitation_stations, "Sum": len(precipitation_stations)
        })

    out = pd.DataFrame(matched).drop_duplicates(subset=["Gewässerstation"])

    out = out[["Gewässer", "Gewässerstation", "Matched precipitation stations", "Sum"]]
    out.to_csv("Matched coordinates.csv", index=False, encoding="utf-8")

    print("Matched the coordinates and stored in table in 'Matched coordinates.csv'.")

matching_the_stations()


Matched the coordinates and stored in table in 'Matched coordinates.csv'.
